In [5]:
#load necessary modules
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.__version__

'1.3.0'

In [6]:
#load dataset
mnist = input_data.read_data_sets("./mnist_data/")
print(mnist.train.num_examples)

Extracting ./mnist_data/train-images-idx3-ubyte.gz
Extracting ./mnist_data/train-labels-idx1-ubyte.gz
Extracting ./mnist_data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/t10k-labels-idx1-ubyte.gz
55000


In [7]:
train_image = mnist.train.images
train_label = mnist.train.labels
test_image = mnist.test.images
test_label = mnist.test.labels

In [20]:
#plot function
def plot_images(input_images, image_labels, image_pre_labels=None):
    assert len(input_images) == 9
    
    fig, axes = plt.subplots(3,3)
    #fig.subplots_adjust(hspace=.3, wspace=.3)
    
    for i, ax in enumerate(axes.flat):
        
        ax.imshow(input_images[i].reshape((28,28)), cmap="binary")
        if image_pre_labels == None:
            xlabel = "true:{0}".format(image_labels[i])
        else:
            xlabel = "true:{0}, predict:{1}".format(iamge_labels[i], image_pre_labels[i])
        ax.set_xlabel(xlabel)
        
        ax.set_xticks([])
        ax.set_yticks([])
        
    plt.show()

In [21]:
plot_images(train_image[:9], train_label[:9])

In [22]:
# parameters for cnn
filter_size = 5
num_filters1 = 16
num_filters2 = 36
fc_size = 128

In [ ]:
#define inputs
batch_size = 100

x = tf.placeholder([None, 784], name="images")
y = tf.placeholder([None, 10], name="real_labels")

In [ ]:
# define model
#layer1 
#define functions

def get_weights(shape):
    return tf.get_variable(initializer=tf.truncated_normal_initializer(shape, stddev=0.5))

def get_bias(shape):
    return tf.get_variable(initializer=tf.constant(0.05, shape=shape))

def conv_layer(inputs, in_channels, out_channels, filter_size, stride = 1, pooling=True):
    input_channel = in_channels
    output_channel = out_channels
    filters = [filter_size, filter_size, input_channel, output_channel]
    weights = get_weights(filters)
    bias = get_bias()
    return tf.nn.conv2d(inputs, filters, stride = stride)

def pooling_layer(inputs, window_size, means = "max"):
    if means == "mean":
        return tf.nn.avg_pool(inputs, ksize=window_size)
    else:
        return tf.nn.max_pool(inputs, ksize=window_size)